In [ ]:
#Function is defined
def Slice_Sequence(path_now, window, step):
    import sys, os
    
    
    #Starts very handy counters
    n1 = 0  #Helps to define the amount of groups we will be working with
    n2 = 0  #Keeps track of the group that we are currently working on
    independently = 0  #keeps track of whether we are coming from Extract_IDRs or working independently
    counter = 0  #keeps track of how many fasta files are present within current group (either .fasta or .txt)
    folder_created = 0  #keeps track of whether a folder was already created within the output destination to store the coded lists
    
    #Creates temporary variables to manage paths
    files = next(os.walk(path_now))[2]  #Helps to check if we are in the main folder generated by Extract_IDRs by compiling a list of all files within this folder
    folders = next(os.walk(path_now))[1]   #Helps to make a list of all the groups and theirs paths that we will be going through by compiling a list of all folders within this folder
    fastas = {}  #Lists all the fasta files for each group (either .fasta or .txt)

    
    #Checks if we are in the main folder generated by Extract_IDRs
    if 'Summary.txt' in files:
        print("Extract_IDRs has been used!"+'\n')
        #Explains where to find output
        print("Output is set to newly created Slices folder within each group's respective folder insisde Extract_IDRs_Results folder!" + "\n")
        for f in folders:
            #Updates folders with the path to the subfolder 'IDRs' where Extract_IDRs stored the fasta sequences of the IDRs
            folders[n1] = path_now + folders[n1] +'/IDRs/'
            #Compiles the fasta sequences of the IDRs for each group within a variable (either .fasta or .txt)
            fastas[n1] = [f for f in os.listdir(folders[n1]) if f.endswith(".txt") | f.endswith(".fasta")]
            n1 = n1 + 1            
            
        #In case we are in the main folder generated by Extract_IDRs, updates output_path to a list of outputs so that each group folder within
        # the Extract_IDRs folder will have their own Slices folder
        output_paths = next(os.walk(path_now))[1]   #Helps to make a list of all the groups and theirs paths that we will be going through by compiling a list of all folders within this folder
        #Resets n1
        n1 = 0        
        for f2 in output_paths:
            output_paths[n1] = path_now + output_paths[n1]
            #Updates output path to aid with folder creation
            output_paths[n1] = output_paths[n1]+"/Slices_of_"+str(window)+"_StepSize_"+str(step)          
            n1 = n1 + 1 
            

    #Checks if we are in a folder containing fasta sequences (either .fasta or .txt), thus we opted not to use Extract_IDRs first
    elif files != []:
        print("Extract_IDRs has not been used!"+'\n')
        #Explains where to find output
        print("Output is set to newly created Slices folder in the same destination as input!" + "\n")
        independently = 1 # we are working independently from Extract_IDRs
        #Compiles the fasta sequences inside the this folder within a variable (either .fasta or .txt)
        fastas[n1] = [f for f in os.listdir(path_now) if f.endswith(".txt") | f.endswith(".fasta")]       
        
        #Checks for error: no fasta files inside specified location (neither .fasta nor .txt)
        if fastas[n1] == []:
            sys.exit("No fasta files were found (neither .fasta nor .txt)! \nPlease, try a different folder! =)") 
        
        #If there is no error, proceeds with code    
        n1 = n1 + 1
        
        #switches folders to path_now so the program can run independently from Extract_IDRs
        if path_now.endswith("/"):
            folders = path_now
        else:
            folders = path_now + "/"
            
        #Defines output path to the same folder where input is, but a folder called slices will be created later
        output_path = path_now
        #Updates output path to aid with folder creation
        output_path = output_path+"/Slices_of_"+str(window)+"_StepSize_"+str(step)
    
    
    #Exits program because no fasta files were found (either .txt or .fasta)
    else:
        sys.exit("No fasta files were found (neither .fasta nor .txt)! \nPlease, try a different folder!") 

        
        
    #Preapares to loop through the groups
    while n2 < n1:
        
        if independently == 0:  #Extract IDRs was used
            #Changes current directory to the folder containing the fasta sequences (either .fasta or .txt)
            os.chdir(folders[n2])
            #Prepares to create a folder named 'Slices' to store the sequence slices inside the folder of the current group
            os.mkdir(output_paths[n2])
            #updates output path to move on with code
            output_paths[n2] = output_paths[n2]+"/"            
            
        if independently == 1:  #Extract IDRs was NOT used 
            #Changes current directory to the folder containing the fasta sequences (either .fasta or .txt)
            os.chdir(folders)
            #Prepares to create a folder named 'Slices' to store the sequence slices inside the folder of the current group
            os.mkdir(output_path)
            #updates output path to move on with code
            output_path = output_path+"/"

        
        #Starts to loop trhough each fasta file within the group to add them to the list file   
        #print(fastas)
        counter = 0  #Resets counter
        for f in fastas[n2]:
            counter = counter + 1
            
            #Creates handy counters/variables
            current_seq_lenth = 0  #for the program to know when to stop
            current_position = 0  #current residue / start of the slice
            slice_end = 0  #end of the slice residue
            n_seq = str(n2)+str(counter)  #keeps track of the current sequence number using group number (n2) and sequence number (counter)
            this_name = '' #Creates/resets variable for current protein name
            seq = '' #Creates/resets variable for current protein sequence
            lines = ''  #Creates/resets variable to store all lines in current fasta file
            positions = ''  #Creates/resets variable to store IDR start and end residue number when coming from Extract IDRs
                      
            
            #Loads sequence into variable seq
            c = open(f)  #Opens current fasta file
            lines = c.readlines()   #Reads all lines into temporary variable lines
            for line in lines:
                if line.startswith ('>') == True:
                    this_name = line   #Extracts protein name into this_name
                    #If Extract IDRs was used
                    if independently == 0:
                        this_name_split = this_name.split("|")   #Splits first line of FASTA file to obtain IDR start and end residue numbers
                        positions = this_name_split[-2].split("_")  #Extracts IDR start and end residue numbers
                if line.startswith ('>') == False:
                    seq = r''.join((seq, line.strip()))  #Extracts protein sequence into seq            
            c.close()

            #Deletes spaces and \n from the sequence so it can be properly processed
            seq = seq.replace(" ", "")
            seq = seq.replace("\n", "")            

            #When not using Extract IDRs
            if independently == 1:
            #Updates current sequence length
                current_seq_length = len(seq)
            #Resets current position to 0 and updates slice_end
                current_position = 0
                slice_end = current_position + window
            
            #When using Extract IDRs
            if independently == 0:
            #Updates current sequence length
                current_seq_length = int(positions[1]) + 1
            #Resets current position to 0 and updates slice_end
                current_position = int(positions[0])
                slice_end = current_position + window


            #Starts to loop through the sequence and extract the slices
            while slice_end <= current_seq_length:

                if independently == 0:  #Extract IDRs was used
                    #program will use output_paths[n2]
                    
                    #Extracts the slice
                    current_slice = seq[(current_position-int(positions[0])):(slice_end-int(positions[0]))]
                    #print(str(current_position + 1) +"  "+ str(current_slice) +"  "+ str(slice_end))
            
                    #For Protein Name, file name: (Preparing to store results)
                    protein_name = this_name.strip("\n")+"|"+str(current_position)+"_"+str(slice_end-1)
                    file_name = f.strip(".txt")+"_"+str(current_position)+"_"+str(slice_end-1)+'.txt'
                    
                    #Saving Results
                    f1 = open(str(output_paths[n2])+str(file_name), "a")
                    f1.write(str(protein_name)+"\n"+str(current_slice))
                    f1.close()
            
                    #For initial residue, slice sequence, final residue
                    #print(str(current_position + 1) +"  "+ str(current_slice) +"  "+ str(slice_end))

                if independently == 1:  #Extract IDRs was NOT used
                    #program will use output_path
                    
                    #Extracts the slice
                    current_slice = seq[current_position:slice_end]
                    #print(str(current_position + 1) +"  "+ str(current_slice) +"  "+ str(slice_end))
                    
                    #For Protein Name, file name: (Preparing to store results)
                    protein_name = this_name.strip("\n")+"|"+str(current_position + 1)+"_"+str(slice_end)
                    file_name = f.strip(".txt")+"_"+str(current_position + 1)+"_"+str(slice_end)+'.txt'
                    
                    #Saving Results
                    f1 = open(str(output_path)+str(file_name), "a")
                    f1.write(str(protein_name)+"\n"+str(current_slice))
                    f1.close()
                    
                    #For initial residue, slice sequence, final residue
                    #print(str(current_position + 1) +"  "+ str(current_slice) +"  "+ str(slice_end))
                
                #Moves to the next iteration
                current_position = current_position + step
                slice_end = slice_end + step
       
    
        #Prepares counter for next iteration
        n2 = n2 + 1
    
    print("\n End \n")  
    
    ##This is what this function returns to be subsequently used in the IDR Sequence Composition Module/Function                                          
    return #output_path
